In [1]:
import pandas as pd #noqa: F401
import numpy as np #noqa: F401

import datetime
from datetime import datetime  ###, timedelta, timezone
import phonenumbers

## profile-<date>.csv is a dump of the Empower data.  I store these on the N: drive
today = '24-9-24'
home = 'N:/'
path = home + 'Al/RelationalVoterProgram/Python/ReadEmpowerData_2024/'
data_file = 'Profiles/profiles-' + today + '.csv'

## or sometimes I keep them locally...
# home = 'C:/'
# path = home + 'Users/nicho/Downloads/'
# data_file = 'profiles-' + today + '.csv'

organizers = ['Director', 'Organizer', 'Volunteer']
contacts = ['Contact']
regions = ['Green Bay', 'Kenosha', 'Racine', 'Madison', 'Milwaukee', 'Manitowoc', 'Sheboygan',
    'Walworth', 'Waukesha', 'Unknown Region']

import sys
sys.path.append('../Common')
from empowerJSONhelpers import InitializeDataFrames, beginningOfTime
from edatools import ColumnMove #noqa: F401

## these are arbitrary cutoffs -- you can use them to look at activity in the RVP since that date. 
## You could pick an earlier date and get more results to work with, but your additional date might
## be so stale as to be irrelevant.

# startDate22 = '2022-01-01'
# campaignStartDate22 = datetime.fromisoformat(startDate22)

# startDate23 = '2023-01-01'
# campaignStartDate23 = datetime.fromisoformat(startDate23)

# startDate24 = '2024-04-01'
# campaignStartDate24 = datetime.fromisoformat(startDate24)


In [2]:
## reformat the time stamps in the Empower data dump and convert them to datetime objects
## for easy comparison.
def SplitTime(theTime):
    if isinstance(theTime, int) or isinstance(theTime, float):
        if np.isnan(theTime):
            return beginningOfTime.date()
        return pd.to_datetime(time.strftime("%Y-%m-%d",time.gmtime(int(theTime/1000)))).date()
    if not theTime or theTime.isspace():
        return beginningOfTime.date()
    if 'T' in theTime:
        date, time = theTime.split('T')
    elif ' ' in theTime:
        date, time = theTime.split(' ')
    date_dt = pd.to_datetime(date)
    return date_dt.date()

def IsBlank(s):
    if isinstance(s, float) or isinstance(s, int): return True
    try:
        return bool(not s or s.isspace())
    except ValueError:
        print(type(s))

def IsNotBlank(s):
    if isinstance(s, float) or isinstance(s, int): return False
    return bool(s and not s.isspace())

def ParsePhoneUS(phone_number):
    default_phone = ''
    if isinstance(phone_number, float) or isinstance(phone_number, int): return default_phone
    assert(isinstance(phone_number, str))
    if phone_number == 'None' or phone_number == 'nan' or IsBlank(phone_number): return default_phone
    if (len(phone_number) > 10 and phone_number[0] == '+'):
        phone_number = phone_number[1:]
    if(len(phone_number) > 10 and phone_number[0] == '1'):
        phone_number = phone_number[1:]
    if len(phone_number) > 10  and phone_number[-1] == '0':
        phone_number = phone_number[:10]
    try:
        my_number = phonenumbers.parse(phone_number, region = 'US')
        return phonenumbers.format_number(my_number, phonenumbers.PhoneNumberFormat.NATIONAL)
    except phonenumbers.NumberParseException:
        return default_phone

def CleanPhone(df, phone_column = 'phone'):
    """
    Convert a ten-digit string to a more readable (xxx) xxx-xxxx format
    Args:
        df (Pandas DataFrame)
        phone_column (str, optional): Name of the column where
        the phone numbers are found. Defaults to 'phone'.

    Returns:
        Inpuyt DataFrame with specified column converted
    """    """"""
    column_list = list(df.columns)
    default_phone = ''
    if phone_column in column_list:
        indx = column_list.index(phone_column)
        df[phone_column] = df[phone_column].astype(str)
        df[phone_column] = df[phone_column].fillna(default_phone)
        df['phone_clean'] = df[phone_column].apply(lambda x : ParsePhoneUS(x))
        df.drop(columns = [phone_column], inplace = True)
        df.rename(columns = {'phone_clean':phone_column}, inplace = True)
        df.insert(indx, phone_column, df.pop(phone_column))
    return df


In [3]:
## download the data
data = InitializeDataFrames(path, data_file, {})
if not data.empty:
    print('Loaded', len(data), 'records.')

data['Created At'] = data['Created At'].apply(lambda x : SplitTime(x)) 
data['Last Used Empower At'] = data['Last Used Empower At'].apply(lambda x : SplitTime(x)) 
data.drop(columns=['Address Line 2'], inplace=True)
data.fillna({'First Name':' ', 'Last Name': ' ', 'Parent EID':' '},  inplace=True)
data = CleanPhone(data, phone_column='Phone')
print("Cleanup complete.")

Loading data from file...
Data loaded, elapsed time: 4.74 seconds.
Loaded 30622 records.
Cleanup complete.


In [4]:
columns_to_keep = ['Parent EID', 'EID', 'Role','First Name', 'Last Name', 
    'Phone', 'Created At','Last Used Empower At']

## subset the leaders and the contacts
leaders = data.loc[data['Role'].isin(organizers)]
contacts = data.loc[data['Role'].isin(contacts)]

## over the years, many leaders signed onto the system once and never did anything. We want to consider only
## people who signed on at least twice, so their last use date is later than their start date.
print('initial leader count:', len(leaders))
activated_leaders = leaders.loc[leaders['Last Used Empower At'] > leaders['Created At']]
print('activated leader count:', len(activated_leaders))

## we only want to consider people for whom we have a phone number we can call
mask = activated_leaders['Phone'].apply(lambda x : IsNotBlank(x))
reachable_activated_leaders = activated_leaders[mask]
print('total reachable activated leader count:', len(reachable_activated_leaders))

reachable_activated_leaders = reachable_activated_leaders[columns_to_keep]
reachable_activated_leaders.rename(columns={'Parent EID': 'ParentEID', 'First Name': 'FirstName', 
    'Last Name': 'LastName', 'Created At': 'CreatedAt', 'Last Used Empower At': 'LastUsedEmpowerAt'}, inplace=True)

## subset directors, organizers, and volunteers
reachable_directors = reachable_activated_leaders.loc[reachable_activated_leaders['Role'] == 'Director']
print('reachable activated director count:', len(reachable_directors))
reachable_organizers = reachable_activated_leaders.loc[reachable_activated_leaders['Role'] == 'Organizer']
print('reachable activated organizer count:', len(reachable_organizers))
reachable_volunteers = reachable_activated_leaders.loc[reachable_activated_leaders['Role'] == 'Volunteer']
print('reachable activated volunteer count:', len(reachable_volunteers))


initial leader count: 1622
activated leader count: 847
total reachable activated leader count: 309
reachable activated director count: 18
reachable activated organizer count: 21
reachable activated volunteer count: 270


In [5]:
parent_names = []
parent_roles = []
full_names = []
for row in reachable_activated_leaders.itertuples():
    parent_eid = row.ParentEID
    full_name = row.FirstName + ' ' + row.LastName
    parent_data = data.loc[data['EID'] == parent_eid]
    try:
        parent_name = parent_data['First Name'].values[0] + ' ' + parent_data['Last Name'].values[0]
        parent_role = parent_data['Role'].values[0]
    except (ValueError, IndexError):
        parent_name = " "
        parent_role = " "
    parent_names.append(parent_name)
    parent_roles.append(parent_role)
    full_names.append(full_name)
    
reachable_activated_leaders['ParentName'] = parent_names
reachable_activated_leaders['ParentRole'] = parent_roles
reachable_activated_leaders['FullName'] = full_names
reachable_activated_leaders = ColumnMove(reachable_activated_leaders, 'ParentRole', 1)
reachable_activated_leaders = ColumnMove(reachable_activated_leaders, 'ParentName', 2)
reachable_activated_leaders = ColumnMove(reachable_activated_leaders, 'FullName', 5)
reachable_activated_leaders.drop(columns=['FirstName', 'LastName'], inplace=True)
reachable_activated_leaders.sort_values(by=['Role','LastUsedEmpowerAt'], ascending = [True, False], inplace=True)
reachable_activated_leaders.reset_index(drop=True, inplace=True)

In [6]:
multi = reachable_activated_leaders.set_index(['ParentRole','ParentEID','ParentName']).sort_values(by = ['ParentRole','ParentName', 'LastUsedEmpowerAt'], ascending = [True, True, False])

In [7]:
## former directors and supervoceros, no longer with the organization
departed_eids = [\
'u-56-3016',  # Al Nichols
'u-56-18093', # Fabi,
'u-56-21557', # Jake Pena
'sZONfFaf3OOHvz', # Gilbert Nunez
'idKoO5r7j1gFxv', # Emily Firlinger
'rboft4TAXwHySN', # Jennifer Estrada
'Ya6XAA9GBhi8Kl', # Dummy Account
'bqUNmQHKI3w5AZ', # Ray Pugh
'FAzpkk9IzXrbt6', # Clay Perry
'u-56-21623',     # Allison Vasquez-Lovell
'1oRrlB5DWlbmxz', # Jose Rivera
'm4pozco9m8exul', # Ruben Ramos
'TBNKT6PmgtvmOB', # Minerva Cornejo
'zwrKe8ICrcIQB2', # Alex Arellano
'2HAXLm7OCK8VHC', # Luam Rincon
'c9r65cbijsca55', # Emily Torres Luevano
'k03oyt4ddgd9au', # Vicky Calderon
'P0CWy9vUnIHhkO', # Leslie Flores
't21hvz9jdl15vt', # Ricardo Torres
'u-56-5837',      # Angel Sanchez
'5esYyfuBwTApq8', # Raquel Alvarado
'fncmu10iwxbded', # Stephanie Salgado Altimarano
'c-167138',       # Ania Jimenez
'u-56-1841',      # Liam Gonzalez
'vIHtcC9AjNewEl', # the Unknown Vocero
'u-56-19305',     # Cassandra Casas
'u-56-21499'] #Yesenia Perez

departed_names = [\
'Alan Nichols',  # Al Nichols
'Fabian Maldonado', # Fabi,
'Jake Pena', # Jake Pena
'Gilbert Nunez', # Gilbert Nunez
'Emily Firlinger', # Emily Firlinger
'Jennifer Estrada', # Jennifer Estrada
'Dummy Account', # Dummy Account
'Ray Pugh', # Ray Pugh
'Clay Perry', # Clay Perry
'Allison Vasquez-Lovell',     # Allison Vasquez-Lovell
'Jose Rivera', # Jose Rivera
'Ruben Ramos', # Ruben Ramos
'Minerva Cornejo', # Minerva Cornejo
'Alex Arellano', # Alex Arellano
'Luam Rincon', # Luam Rincon
'Emily Torres Luevano', # Emily Torres Luevano
'Vicky Calderon', # Vicky Calderon
'Leslie Flores', # Leslie Flores
'Ricardo Torres',
'Angel Sanchez',
'Raquel Alvarado',
'Stephanie Salgado Altimarano',
'Ania Jimenez',
'Liam Gonzalez',
'the Unknown Vocero',
'Cassandra Casas',
'Yesenia Perez' ]

def highlight_cells(s):
    return "background-color: yellow;" if s in departed_eids or s.strip() in departed_names else ""

with pd.ExcelWriter('test.xlsx' ) as writer:
    workbook = writer.book
    worksheet = workbook.create_sheet('Sheet1')
    multi.style.map_index(highlight_cells, axis = 'index', level = [1,2] ).to_excel(writer, "Sheet1",  engine='xlsxwriter')